In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import dash
from dash import dcc, html, dash_table
import plotly.express as px
import matplotlib.pyplot as plt
import io
import base64

DB_PATH = 'G:/Meu Drive/Documents/GitHubPublished/DataScienceProject/database/ecommerceProject.db'
conn = None  

def run_query(query: str, params=()):
    """
    Executa uma consulta SQL no banco de dados e retorna o resultado como um DataFrame.
    Adiciona tratamento de erro para a execução da query.
    """
    global conn
    if conn is None:
        try:
            conn = sqlite3.connect(DB_PATH)
        except sqlite3.Error as e:
            print(f"Erro ao conectar ao banco de dados: {e}")
            return pd.DataFrame() 

    try:
        df = pd.read_sql(query, conn, params=params)
        return df
    except Exception as e:
        print(f"Erro ao executar a query: {query}\nErro: {e}")
        return pd.DataFrame() 

def load_data():
    """
    Carrega todos os DataFrames necessários para a análise usando consultas SQL.
    """
    queries = {
        "customers": 'SELECT * FROM customers',
        "orders": 'SELECT * FROM orders',
        "products": 'SELECT * FROM products',
        "categories": 'SELECT * FROM categories',
        "order_items": 'SELECT * FROM order_items',
        "products_by_categories": '''
            SELECT c.category_name, COUNT(DISTINCT p.product_name) AS count
            FROM categories c
            JOIN products p ON c.category_id = p.category_id
            GROUP BY c.category_name
            ORDER BY c.category_name''',
        "top_10_customer_value": '''
            SELECT c.customer_name, ROUND(SUM(o.order_value),2) AS total
            FROM customers c
            JOIN orders o ON c.customer_id = o.customer_id
            GROUP BY c.customer_name
            ORDER BY total DESC
            LIMIT 10''',
        "sales_over_time": '''
            SELECT STRFTIME('%Y-%m', order_date) AS month, SUM(order_value) AS total_sales
            FROM orders
            GROUP BY month
            ORDER BY month''',
        "top_selling_products_quantity": '''
            SELECT p.product_name, SUM(oi.order_quantity) AS total_quantity_sold
            FROM products p
            JOIN order_items oi ON p.product_id = oi.product_id
            GROUP BY p.product_name
            ORDER BY total_quantity_sold DESC
            LIMIT 10''',
        "order_value_vs_items": '''
            SELECT o.order_value, COUNT(oi.items_id) AS num_items
            FROM orders o
            JOIN order_items oi ON o.order_id = oi.order_id
            GROUP BY o.order_id, o.order_value''',
        "products_without_category": '''
            SELECT p.product_id, p.product_name
            FROM products p
            LEFT JOIN categories c ON p.category_id = c.category_id
            WHERE c.category_id IS NULL''',
        "orders_with_invalid_customer": '''
            SELECT o.order_id, o.customer_id
            FROM orders o
            LEFT JOIN customers c ON o.customer_id = c.customer_id
            WHERE c.customer_id IS NULL'''
    }
    
    data = {}
    for key, query in queries.items():
        data[key] = run_query(query)
    return data

def fig_to_base64(fig):
    """
    Converte uma figura Matplotlib em uma string base64 para ser exibida em HTML.
    """
    buf = io.BytesIO()
    fig.savefig(buf, format="png", bbox_inches='tight')
    plt.close(fig)
    return base64.b64encode(buf.getvalue()).decode()

# --- Funções de Validação de Dados ---
def generate_quality_report(df, df_name, unique_id_col=None, numeric_cols=None):
    """
    Gera um relatório HTML com métricas de qualidade de dados para um DataFrame.
    Inclui valores ausentes, duplicados e uma análise básica de outliers.
    """
    if df.empty:
        return html.Div([
            html.H4(f"Relatório de Qualidade de Dados para {df_name}", style={'color': '#333'}),
            html.P("DataFrame vazio. Verifique a fonte de dados.", style={'color': 'red'})
        ], style={'marginBottom': '20px', 'border': '1px solid #ddd', 'padding': '15px', 'borderRadius': '5px'})

    report_items = [html.H4(f"Relatório de Qualidade de Dados para {df_name}", style={'color': '#333'})]
    
    # 1. Valores Ausentes
    missing_data = df.isnull().sum()
    missing_data = missing_data[missing_data > 0]
    if not missing_data.empty:
        report_items.append(html.P("Valores Ausentes:", style={'fontWeight': 'bold'}))
        for col, count in missing_data.items():
            report_items.append(html.Li(f"'{col}': {count} valores ausentes ({count/len(df)*100:.2f}%)"))
    else:
        report_items.append(html.P("Nenhum valor ausente encontrado.", style={'color': 'green'}))
        
    # 2. Linhas Duplicadas
    duplicates = df.duplicated().sum()
    if duplicates > 0:
        report_items.append(html.P(f"Linhas Duplicadas (total): {duplicates}", style={'color': 'orange', 'fontWeight': 'bold'}))
    else:
        report_items.append(html.P("Nenhuma linha duplicada encontrada (total).", style={'color': 'green'}))

    # 3. Duplicatas baseadas em ID único
    if unique_id_col and unique_id_col in df.columns:
        id_duplicates = df.duplicated(subset=[unique_id_col]).sum()
        if id_duplicates > 0:
            report_items.append(html.P(f"Linhas duplicadas baseadas no '{unique_id_col}': {id_duplicates}", style={'color': 'red', 'fontWeight': 'bold'}))
        else:
            report_items.append(html.P(f"Nenhuma duplicata para '{unique_id_col}'.", style={'color': 'green'}))

    # 4. Análise de Outliers (Método IQR) para colunas numéricas especificadas
    if numeric_cols:
        report_items.append(html.P("Outliers (Método IQR):", style={'fontWeight': 'bold'}))
        outliers_found = False
        for col in numeric_cols:
            if col in df.columns and pd.api.types.is_numeric_dtype(df[col]) and len(df[col].dropna()) > 0:
                Q1 = df[col].quantile(0.25)
                Q3 = df[col].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR
                outliers_count = df[(df[col] < lower_bound) | (df[col] > upper_bound)].shape[0]
                if outliers_count > 0:
                    report_items.append(html.Li(f"'{col}': {outliers_count} outliers ({outliers_count/len(df)*100:.2f}%)", style={'color': 'orange'}))
                    outliers_found = True
        if not outliers_found:
            report_items.append(html.Li("Nenhum outlier significativo detectado nas colunas numéricas fornecidas.", style={'color': 'green'}))
    
    return html.Div(report_items, style={'marginBottom': '20px', 'border': '1px solid #ddd', 'padding': '15px', 'borderRadius': '5px', 'backgroundColor': '#f9f9f9'})

# --- Carregar os Dados ---
dataframes = load_data()

# Atribuir os DataFrames (com pd.DataFrame() vazio para evitar erros caso a query falhe)
tCustomers = dataframes.get("customers", pd.DataFrame())
tOrders = dataframes.get("orders", pd.DataFrame())
tProducts = dataframes.get("products", pd.DataFrame())
tCategories = dataframes.get("categories", pd.DataFrame())
tOrderItens = dataframes.get("order_items", pd.DataFrame())
tProductsByCategories = dataframes.get("products_by_categories", pd.DataFrame())
tTop10CustomerValue = dataframes.get("top_10_customer_value", pd.DataFrame())
tSalesOverTime = dataframes.get("sales_over_time", pd.DataFrame())
tTopSellingProductsQuantity = dataframes.get("top_selling_products_quantity", pd.DataFrame())
tOrderValueVsItems = dataframes.get("order_value_vs_items", pd.DataFrame())
tProductsWithoutCategory = dataframes.get("products_without_category", pd.DataFrame())
tOrdersWithInvalidCustomer = dataframes.get("orders_with_invalid_customer", pd.DataFrame())


# --- Criação de Gráficos (Plotly Express para interatividade) ---

# Quantidade de Produtos por Categoria
fig_products_by_category = px.bar(
    tProductsByCategories, x='category_name', y='count',
    title='Quantidade de Produtos por Categoria',
    labels={'category_name': 'Categoria', 'count': 'Quantidade'},
    color='count', color_continuous_scale=px.colors.sequential.PuBu
)

# Top 10 Clientes por Valor Total de Compras
fig_top_customers = px.bar(
    tTop10CustomerValue, x='customer_name', y='total',
    title='Top 10 Clientes por Valor Total de Compras',
    labels={'customer_name': 'Cliente', 'total': 'Valor Total'},
    color='total', color_continuous_scale=px.colors.sequential.Greens
)

# Tendência de Vendas Mensais (se houver dados de pedidos)
if not tSalesOverTime.empty:
    fig_sales_time = px.line(
        tSalesOverTime, x='month', y='total_sales',
        title='Tendência de Vendas Mensais',
        labels={'month': 'Mês', 'total_sales': 'Total de Vendas'},
        markers=True
    )
else:
    fig_sales_time = go.Figure().add_annotation(text="Sem dados para tendência de vendas.", showarrow=False)

# Top 10 Produtos Mais Vendidos (por Quantidade)
if not tTopSellingProductsQuantity.empty:
    fig_top_products_quantity = px.bar(
        tTopSellingProductsQuantity, x='product_name', y='total_quantity_sold',
        title='Top 10 Produtos Mais Vendidos (por Quantidade)',
        labels={'product_name': 'Produto', 'total_quantity_sold': 'Quantidade Vendida'},
        color='total_quantity_sold', color_continuous_scale=px.colors.sequential.Oranges
    )
else:
    fig_top_products_quantity = go.Figure().add_annotation(text="Sem dados de produtos mais vendidos.", showarrow=False)


# Correlação: Valor do Pedido vs. Número de Itens
if not tOrderValueVsItems.empty:
    fig_order_correlation = px.scatter(
        tOrderValueVsItems, x='num_items', y='order_value',
        title='Correlação: Valor do Pedido vs. Número de Itens',
        labels={'num_items': 'Número de Itens no Pedido', 'order_value': 'Valor do Pedido'},
        hover_data=['order_value', 'num_items']
    )
else:
    fig_order_correlation = go.Figure().add_annotation(text="Sem dados para correlação de pedidos.", showarrow=False)


# --- Inicialização do Aplicativo Dash ---
app = dash.Dash(__name__)

# Define o layout do aplicativo Dash
app.layout = html.Div([
    html.H1("Análise Exploratória de Dados e Qualidade - E-commerce", 
            style={"textAlign": "center", "marginBottom": "40px", "color": "#2c3e50"}),

    # --- Relatórios de Qualidade de Dados ---
    html.H2("Relatórios de Qualidade de Dados", style={"marginTop": "50px", "color": "#34495e"}),
    html.Div([
        generate_quality_report(tCustomers, "Clientes", unique_id_col='customer_id'),
        generate_quality_report(tOrders, "Pedidos", unique_id_col='order_id', numeric_cols=['order_value']),
        generate_quality_report(tProducts, "Produtos", unique_id_col='product_id', numeric_cols=['price']),
        generate_quality_report(tCategories, "Categorias", unique_id_col='category_id'),
        generate_quality_report(tOrderItens, "Itens de Pedido", numeric_cols=['quantity', 'price_at_order'])
    ], style={'display': 'flex', 'flexWrap': 'wrap', 'gap': '20px', 'justifyContent': 'space-around'}),

    # --- Inconsistências Identificadas ---
    html.H2("Possíveis Inconsistências", style={"marginTop": "50px", "color": "#34495e"}),
    html.Div([
        html.Div([
            html.H4("Produtos sem Categoria Válida", style={'color': '#e74c3c'}),
            dash_table.DataTable(
                id='products-without-category-table',
                columns=[{"name": col, "id": col} for col in tProductsWithoutCategory.columns],
                data=tProductsWithoutCategory.to_dict('records'),
                style_table={'overflowX': 'auto', 'border': '1px solid #ddd'},
                style_cell={'textAlign': 'left', 'padding': '10px'},
                style_header={'backgroundColor': '#f8f8f8', 'fontWeight': 'bold'}
            ) if not tProductsWithoutCategory.empty else html.P("Nenhum produto sem categoria válida detectado.", style={'color': 'green'})
        ], style={'width': '48%', 'minWidth': '300px', 'marginBottom': '20px', 'border': '1px solid #ddd', 'padding': '15px', 'borderRadius': '5px'}),
        
        html.Div([
            html.H4("Pedidos com Cliente Inválido/Inexistente", style={'color': '#e74c3c'}),
            dash_table.DataTable(
                id='orders-with-invalid-customer-table',
                columns=[{"name": col, "id": col} for col in tOrdersWithInvalidCustomer.columns],
                data=tOrdersWithInvalidCustomer.to_dict('records'),
                style_table={'overflowX': 'auto', 'border': '1px solid #ddd'},
                style_cell={'textAlign': 'left', 'padding': '10px'},
                style_header={'backgroundColor': '#f8f8f8', 'fontWeight': 'bold'}
            ) if not tOrdersWithInvalidCustomer.empty else html.P("Nenhum pedido com cliente inválido detectado.", style={'color': 'green'})
        ], style={'width': '48%', 'minWidth': '300px', 'marginBottom': '20px', 'border': '1px solid #ddd', 'padding': '15px', 'borderRadius': '5px'})
    ], style={'display': 'flex', 'flexWrap': 'wrap', 'gap': '20px', 'justifyContent': 'space-around'}),

    # --- Visão Geral das Tabelas ---
    html.H2("Visão Geral das Tabelas (Primeiras 5 linhas)", style={"marginTop": "50px", "color": "#34495e"}),
    html.Details([
        html.Summary("Clique para expandir/recolher as tabelas"),
        html.Div([
            html.H4("Clientes", style={'color': '#2c3e50'}),
            dash_table.DataTable(
                id='customers-table',
                columns=[{"name": col, "id": col} for col in tCustomers.columns],
                data=tCustomers.head(5).to_dict('records'),
                style_table={'overflowX': 'auto', 'marginBottom': '20px'},
                style_cell={'textAlign': 'left', 'minWidth': '80px', 'width': '120px', 'maxWidth': '180px'},
                style_data_conditional=[
                    {'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)'}
                ],
                style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'}
            ),
            html.H4("Pedidos", style={'color': '#2c3e50'}),
            dash_table.DataTable(
                id='orders-table',
                columns=[{"name": col, "id": col} for col in tOrders.columns],
                data=tOrders.head(5).to_dict('records'),
                style_table={'overflowX': 'auto', 'marginBottom': '20px'},
                style_cell={'textAlign': 'left', 'minWidth': '80px', 'width': '120px', 'maxWidth': '180px'},
                style_data_conditional=[
                    {'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)'}
                ],
                style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'}
            ),
            html.H4("Produtos", style={'color': '#2c3e50'}),
            dash_table.DataTable(
                id='products-table',
                columns=[{"name": col, "id": col} for col in tProducts.columns],
                data=tProducts.head(5).to_dict('records'),
                style_table={'overflowX': 'auto', 'marginBottom': '20px'},
                style_cell={'textAlign': 'left', 'minWidth': '80px', 'width': '120px', 'maxWidth': '180px'},
                style_data_conditional=[
                    {'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)'}
                ],
                style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'}
            ),
            html.H4("Categorias", style={'color': '#2c3e50'}),
            dash_table.DataTable(
                id='categories-table',
                columns=[{"name": col, "id": col} for col in tCategories.columns],
                data=tCategories.head(5).to_dict('records'),
                style_table={'overflowX': 'auto', 'marginBottom': '20px'},
                style_cell={'textAlign': 'left', 'minWidth': '80px', 'width': '120px', 'maxWidth': '180px'},
                style_data_conditional=[
                    {'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)'}
                ],
                style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'}
            ),
            html.H4("Itens de Pedido", style={'color': '#2c3e50'}),
            dash_table.DataTable(
                id='order-items-table',
                columns=[{"name": col, "id": col} for col in tOrderItens.columns],
                data=tOrderItens.head(5).to_dict('records'),
                style_table={'overflowX': 'auto', 'marginBottom': '20px'},
                style_cell={'textAlign': 'left', 'minWidth': '80px', 'width': '120px', 'maxWidth': '180px'},
                style_data_conditional=[
                    {'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)'}
                ],
                style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'}
            )
        ])
    ], style={'marginBottom': '40px', 'border': '1px solid #ccc', 'padding': '15px', 'borderRadius': '8px', 'boxShadow': '2px 2px 5px rgba(0,0,0,0.1)'}),

    # --- Visualizações Gráficas para Tendências, Correlações e Outliers ---
    html.H2("Tendências, Correlações e Outliers", style={"marginTop": "50px", "color": "#34495e"}),
    
    html.Div([
        html.H4("Quantidade de Produtos por Categoria", style={'color': '#2c3e50'}),
        dcc.Graph(figure=fig_products_by_category)
    ], style={'marginBottom': '40px', 'border': '1px solid #eee', 'borderRadius': '8px', 'padding': '10px', 'boxShadow': '1px 1px 3px rgba(0,0,0,0.05)'}),

    html.Div([
        html.H4("Top 10 Clientes por Valor Total de Compras", style={'color': '#2c3e50'}),
        dcc.Graph(figure=fig_top_customers)
    ], style={'marginBottom': '40px', 'border': '1px solid #eee', 'borderRadius': '8px', 'padding': '10px', 'boxShadow': '1px 1px 3px rgba(0,0,0,0.05)'}),

    html.Div([
        html.H4("Tendência de Vendas Mensais", style={'color': '#2c3e50'}),
        dcc.Graph(figure=fig_sales_time)
    ], style={'marginBottom': '40px', 'border': '1px solid #eee', 'borderRadius': '8px', 'padding': '10px', 'boxShadow': '1px 1px 3px rgba(0,0,0,0.05)'}),

    html.Div([
        html.H4("Top 10 Produtos Mais Vendidos (por Quantidade)", style={'color': '#2c3e50'}),
        dcc.Graph(figure=fig_top_products_quantity)
    ], style={'marginBottom': '40px', 'border': '1px solid #eee', 'borderRadius': '8px', 'padding': '10px', 'boxShadow': '1px 1px 3px rgba(0,0,0,0.05)'}),

    html.Div([
        html.H4("Correlação: Valor do Pedido vs. Número de Itens", style={'color': '#2c3e50'}),
        dcc.Graph(figure=fig_order_correlation)
    ], style={'marginBottom': '40px', 'border': '1px solid #eee', 'borderRadius': '8px', 'padding': '10px', 'boxShadow': '1px 1px 3px rgba(0,0,0,0.05)'}),
])

if __name__ == '__main__':
    try:
        app.run(debug=True)
    finally:
        # Garante que a conexão seja fechada quando o aplicativo Dash encerrar
        if conn:
            conn.close()
            print("Conexão com o banco de dados SQLite fechada.")

Conexão com o banco de dados SQLite fechada.


In [ ]:
# Executa as consultas para obter os DataFrames
tCustomers = runQuery("queryCustomers")
tOrders = runQuery("queryOrders")
tProducts = runQuery("queryProducts")
tCategories = runQuery("queryCategories")
tOrderItens = runQuery("queryOrderItens")
tCountProducts = runQuery("queryCountProducts")
tCountCategories = runQuery("queryCountCategories")
tAvgSales = runQuery("queryAvgSales")
tDistinctCategories = runQuery("queryDistinctCategories")
tProductsByCategories = runQuery("queryProductsByCategories")
tTop10CustomerValue = runQuery("queryTop10CustomerValue")

# Inicia o aplicativo Dash
app = dash.Dash(__name__)

# Define o layout com tabelas interativas
app.layout = html.Div([
    html.H1("Tabelas do Banco de Dados E-commerce (Consultas Básicas)", style={"textAlign": "center"}),
    
    html.H2("Clientes"),
    dash_table.DataTable(
        id='customers-table',
        columns=[{"name": col, "id": col} for col in tCustomers.columns],
        data=tCustomers.to_dict('records'),
        style_table={'height': 'auto', 'overflowY': 'auto'},
        style_cell={ 'textAlign': 'left'}
    ),
    html.H2("Pedidos"),
    dash_table.DataTable(
        id='orders-table',
        columns=[{"name": col, "id": col} for col in tOrders.columns],
        data=tOrders.to_dict('records'),
        style_table={'height': 'auto', 'overflowY': 'auto'},
        style_cell={ 'textAlign': 'left'}
    ),
    html.H2("Produtos"),
    dash_table.DataTable(
        id='products-table',
        columns=[{"name": col, "id": col} for col in tProducts.columns],
        data=tProducts.to_dict('records'),
        style_table={'height': 'auto', 'overflowY': 'auto'},
        style_cell={ 'textAlign': 'left'}
    ),
    html.H2("Categorias"),
    dash_table.DataTable(
        id='categories-table',
        columns=[{"name": col, "id": col} for col in tCategories.columns],
        data=tCategories.to_dict('records'),
        style_table={'height': 'auto', 'overflowY': 'auto'},
        style_cell={ 'textAlign': 'left'}
    ),
    html.H2("Itens de Pedido"),
    dash_table.DataTable(
        id='order-itens-table',
        columns=[{"name": col, "id": col} for col in tOrderItens.columns],
        data=tOrderItens.to_dict('records'),
        style_table={'height': 'auto', 'overflowY': 'auto'},
        style_cell={ 'textAlign': 'left'}
    ),
    html.H2("Contagem de Produtos"),
    dash_table.DataTable(
        id='count-products-table',
        columns=[{"name": col, "id": col} for col in tCountProducts.columns],
        data=tCountProducts.to_dict('records'),
        style_table={'height': 'auto', 'overflowY': 'auto'},
        style_cell={ 'textAlign': 'left'}
    ),
    html.H2("Contagem de Categorias"),
    dash_table.DataTable(
        id='count-categories-table',
        columns=[{"name": col, "id": col} for col in tCountCategories.columns],
        data=tCountCategories.to_dict('records'),
        style_table={'height': 'auto', 'overflowY': 'auto'},
        style_cell={ 'textAlign': 'left'}
    ),
    html.H2("Média de Vendas"),
    dash_table.DataTable(
        id='avg-sales-table',
        columns=[{"name": col, "id": col} for col in tAvgSales.columns],
        data=tAvgSales.to_dict('records'),
        style_table={'height': 'auto', 'overflowY': 'auto'},
        style_cell={ 'textAlign': 'left'}
    ),
    html.H2("Categorias Distintas"),
    dash_table.DataTable(
        id='distinct-categories-table',
        columns=[{"name": col, "id": col} for col in tDistinctCategories.columns],
        data=tDistinctCategories.to_dict('records'),
        style_table={'height': 'auto', 'overflowY': 'auto'},
        style_cell={ 'textAlign': 'left'}
    ),

    html.H1("Tabelas do Banco de Dados E-commerce (Agregações, Agrupamentos e Joins)", style={"textAlign": "center"}),
    html.H2("Produtos por Categorias"),
    dash_table.DataTable(
        id='products-by-categories-table',
        columns=[{"name": col, "id": col} for col in tProductsByCategories.columns],
        data=tProductsByCategories.to_dict('records'),
        style_table={'height': 'auto', 'overflowY': 'auto'},
        style_cell={ 'textAlign': 'left'}
    ),
    html.H2("Top 10 Clientes por Valor de Compra"),
    dash_table.DataTable(
        id='top10-customer-value-table',
        columns=[{"name": col, "id": col} for col in tTop10CustomerValue.columns],
        data=tTop10CustomerValue.to_dict('records'),
        style_table={'height': 'auto', 'overflowY': 'auto'},
        style_cell={ 'textAlign': 'left'}
    ),
])

if __name__ == '__main__':
    app.run(debug=True)  

conn.close()  # Fecha a conexão com o banco de dados ao final

In [ ]:
"""
Desafio: Relatórios Avançados e Dashboard de Vendas
Agora que você já armazenou os dados no SQLite, o desafio será consultar e analisar os dados diretamente via SQL,
além de criar um dashboard interativo para visualização.

# Parte 1 - Consultas SQL Avançadas
1. Clientes mais frequentes 🔁
    a. Liste os 10 clientes que mais compraram (quantidade de pedidos realizados).
2. Produtos mais rentáveis 💰
    a. Encontre os 5 produtos que mais geraram receita total.
3. Média de valor gasto por categoria 📊
    a. Liste as categorias de produtos com a média de valor gasto por pedido.
4. Comparação de faturamento 📅
    a.Compare o faturamento entre o primeiro semestre (Jan-Jun) e o segundo semestre (Jul-Dez) de 2024.

# Parte 2 - Dashboard Interativo com Streamlit
Agora, crie um dashboard interativo usando Streamlit para visualizar os dados de forma dinâmica.

1. Componentes do Dashboard:
    a. Filtro por mês 📆 (para visualizar as vendas de um período específico).
    b. Gráficos interativos 📊:
        I. Gráfico de barras com os produtos mais vendidos.
        II. Gráfico de linha mostrando o faturamento mensal.
        III. Tabela interativa com os clientes VIPs.
"""

import pandas as pd  # Importa a biblioteca Pandas para manipulação de dados
import sqlite3  # Importa a biblioteca SQLite para interação com bancos de dados SQLite
import streamlit as st  # Importa a biblioteca Streamlit para criação de aplicativos web

conn = sqlite3.connect('salesEcommerce.db')  # Conecta ao banco de dados SQLite 'salesEcommerce.db'
cursor = conn.cursor()  # Cria um cursor para executar comandos SQL

# Define um dicionário com consultas SQL para diferentes análises
dicQueries = {
    "queryTopCustomers": '''
        SELECT customer, SUM(sales_quantity) AS sum_quantity
        FROM sales
        GROUP BY customer
        ORDER BY SUM(sales_quantity) DESC
        LIMIT 10
    ''',  # Consulta para obter os 10 clientes que mais compraram em quantidade
    "queryTopProducts": '''
        SELECT product, SUM(total_price) AS total_revenue
        FROM sales
        GROUP BY product
        ORDER BY SUM(total_price) DESC
        LIMIT 5
    ''',  # Consulta para obter os 5 produtos que geraram mais receita
    "queryAvgPriceCategory": '''
        SELECT category, ROUND(AVG(total_price), 2) AS avg_price
        FROM sales
        GROUP BY category
        ORDER BY AVG(total_price) DESC
    ''',  # Consulta para obter o preço médio gasto por categoria
    "queryHalfYear": '''
        SELECT CASE 
            WHEN STRFTIME('%m', sales_date) BETWEEN '01' AND '06' THEN 'first half-year'
            WHEN STRFTIME('%m', sales_date) BETWEEN '07' AND '12' THEN 'second half-year'
        END AS half_year, ROUND(SUM(total_price), 2) AS total_revenue
        FROM sales
        WHERE STRFTIME('%Y', sales_date) = '2024'
        GROUP BY half_year
        ORDER BY half_year
    '''  # Consulta para comparar o faturamento entre o primeiro e o segundo semestre de 2024
}

def runQuery(queryKey, params=()):
    """Executa uma consulta SQL e retorna o resultado como um DataFrame.

    Args:
        queryKey (str): Chave da consulta no dicionário `dicQueries`.
        params (tuple, optional): Parâmetros para a consulta SQL. Padrão é ().

    Returns:
        pandas.DataFrame: DataFrame com o resultado da consulta.
    """
    query = dicQueries[queryKey]  # Obtém a consulta SQL do dicionário
    return pd.read_sql(query, conn, params=params)  # Executa a consulta e retorna o resultado como um DataFrame

dfTopCustomers = runQuery("queryTopCustomers")  # Executa a consulta para obter os top 10 clientes
dfTopProducts = runQuery("queryTopProducts")  # Executa a consulta para obter os top 5 produtos
dfAvgPriceCategory = runQuery("queryAvgPriceCategory")  # Executa a consulta para obter o preço médio por categoria
dfHalfYear = runQuery("queryHalfYear")  # Executa a consulta para comparar o faturamento por semestre

print("Top 10 Clientes:")
print(dfTopCustomers)  # Imprime o DataFrame com os top 10 clientes

print("\nTop 5 Produtos mais rentáveis:")
print(dfTopProducts)  # Imprime o DataFrame com os top 5 produtos

print("\nMédia de valor gasto por categoria:")
print(dfAvgPriceCategory)  # Imprime o DataFrame com o preço médio por categoria

print("\nComparação de faturamento Jan-Jun e Jul-Dez:")
print(dfHalfYear)  # Imprime o DataFrame com a comparação de faturamento por semestre


st.title(" Sales Dashboard - E-commerce")  # Define o título do aplicativo Streamlit

monthFilter = st.selectbox("Selecione um mês:", ["Todos"] + [f"{m:02d}/2024" for m in range(1, 13)])  # Cria um selectbox para filtrar os dados por mês

# Define um dicionário com consultas SQL para visualização dos dados
dicQueriesViz = {
    "queryBarTopProducts": '''
        SELECT product, SUM(sales_quantity) AS total_quantity
        FROM sales
        {whereClause}
        GROUP BY product
        ORDER BY SUM(sales_quantity) DESC
        LIMIT 10
    ''',  # Consulta para obter os 10 produtos mais vendidos em quantidade
    "queryLineMonthRevenue": '''
        SELECT STRFTIME('%m', sales_date) AS month, SUM(total_price) AS total_revenue
        FROM sales
        {whereClause}
        GROUP BY STRFTIME('%m', sales_date)
        ORDER BY STRFTIME('%m', sales_date)
    ''',  # Consulta para obter o faturamento mensal
    "queryVipCustomers": '''
        SELECT customer, SUM(sales_quantity) AS total_quantity, SUM(total_price) AS total_amount
        FROM sales
        {whereClause}
        GROUP BY customer
        ORDER BY SUM(total_price) DESC, SUM(sales_quantity) DESC, customer
        LIMIT 10
    '''  # Consulta para obter os 10 clientes VIPs (que mais compraram em valor e quantidade)
}

whereClause = ""  # Inicializa a cláusula WHERE da consulta SQL
params = ()  # Inicializa os parâmetros da consulta SQL
if monthFilter != "Todos":  # Verifica se o filtro de mês foi selecionado
    selectMonth = monthFilter.split("/")[0]  # Obtém o número do mês selecionado
    whereClause = "WHERE STRFTIME('%m', sales_date) = ? AND STRFTIME('%Y', sales_date) = '2024'"  # Define a cláusula WHERE para filtrar os dados pelo mês selecionado
    params = (selectMonth,)  # Define os parâmetros da consulta SQL

def runQueryViz(queryKey):
    """Executa uma consulta SQL para visualização e retorna o resultado como um DataFrame.

    Args:
        queryKey (str): Chave da consulta no dicionário `dicQueriesViz`.

    Returns:
        pandas.DataFrame: DataFrame com o resultado da consulta.
    """
    query = dicQueriesViz[queryKey].format(whereClause=whereClause)  # Obtém a consulta SQL do dicionário e formata com a cláusula WHERE
    return pd.read_sql(query, conn, params=params)  # Executa a consulta e retorna o resultado como um DataFrame

st.write("Gráfico de barras com os produtos mais vendidos.")  # Exibe um texto no aplicativo
dfBarTopProducts = runQueryViz("queryBarTopProducts")  # Executa a consulta para obter os top 10 produtos mais vendidos
if not dfBarTopProducts.empty:  # Verifica se o DataFrame não está vazio
    st.bar_chart(dfBarTopProducts.set_index('product'), x_label="Products", y_label="Quantity")  # Cria um gráfico de barras com os top 10 produtos mais vendidos
else:
    st.write("Nenhum dado disponível para o período selecionado.")  # Exibe uma mensagem caso não haja dados para o período selecionado

st.write("Gráfico de linha faturamento mensal.")  # Exibe um texto no aplicativo
dfLineMonthRevenue = runQueryViz("queryLineMonthRevenue")  # Executa a consulta para obter o faturamento mensal
if not dfLineMonthRevenue.empty:  # Verifica se o DataFrame não está vazio
    st.line_chart(dfLineMonthRevenue.set_index('month'), x_label="Months", y_label="Revenue")  # Cria um gráfico de linha com o faturamento mensal
else:
    st.write("Nenhum dado disponível para o período selecionado.")  # Exibe uma mensagem caso não haja dados para o período selecionado

st.write("Tabela interativa com os clientes VIPs")  # Exibe um texto no aplicativo
dfVipCustomers = runQueryViz("queryVipCustomers")  # Executa a consulta para obter os clientes VIPs
if not dfVipCustomers.empty:  # Verifica se o DataFrame não está vazio
    st.dataframe(dfVipCustomers.style.highlight_max(subset=['total_quantity', 'total_amount']), hide_index=True)  # Cria uma tabela interativa com os clientes VIPs, destacando os maiores valores
else:
    st.write("Nenhum dado disponível para o período selecionado.")  # Exibe uma mensagem caso não haja dados para o período selecionado

conn.close()  # Fecha a conexão com o banco de dados